In [1]:
import requests
from bs4 import BeautifulSoup
from csv import DictWriter

def fetch(url):
    try:
        src = requests.get(url)
        data = BeautifulSoup(src.text,'html.parser')
        return data
    except:
        return None

def fetch_403(url):
    try:
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
        src = requests.get(url,headers=headers)
        data = BeautifulSoup(src.text,'html.parser')
        return data
    except:
        return None

def extract(url):
    try:
        d = {}
        url1 = url+"fullcredits"
        data1 = fetch(url1)
        d['title'] = data1.h3.a.text
        d['director'] = data1.find('tbody').find('td',class_='name').text.replace('\n','')[1:-1]
        d['writer'] = []
        x = data1.find_all('table',class_='simpleTable simpleCreditsTable')
        x = x[1].tbody.find_all('tr')
        for i in x:
            for j in i.find_all('td',class_='name'):
                d['writer'].append(j.a.text.replace('\n','')[1:-1])
        d['cast'] = []
        for i in data1.find('table',class_='cast_list').find_all('tr')[1:]:
            try:
                d['cast'].append(i.a.img['title'])
            except:
                continue
        d['storyline'] = []
        url4 = url+'plotsummary'
        data4 = fetch_403(url4)
        data4 = data4.find('div',class_='ipc-html-content-inner-div').text
        d['storyline'] = data4
        d['keywords'] = []
        url2 = url+'keywords'
        data2 =  fetch_403(url2)
        data2 = data2.find('section',class_='ipc-page-section ipc-page-section--base').ul.find_all('li')
        for i in data2:
            try:
                d['keywords'].append(i.a.text)
            except:
                continue
        url5 = url+'ratings'
        data5 = fetch_403(url5)
        d['avg_rating'] = data5.find('span',class_="sc-5931bdee-1 jUnWeS").text
        d['total'] = data5.find('div',class_='sc-5931bdee-3 dWymrF').text
        d['unweighted'] = data5.find('p',class_='sc-32706d9c-1 dRWuAv').text[:3]
        d['ratings'] = []
        url3 = url+'reviews'
        data3 = fetch(url3)
        data3 = data3.find('div',class_='lister-list').find_all('div',class_='lister-item mode-detail imdb-user-review collapsable')
        for i in data3:
            try:
                j = i
                i = i.find('div',class_='ipl-ratings-bar').text.replace('\n','')
                j = j.find('span',class_='display-name-link').a.text
                d['ratings'].append([i,j])
            except:
                continue
        print(d)
        return d
    except:
        return None

def load(d):
    try:
        with open('/Users/karth/OneDrive/Desktop/OELP/Data/DataSet.csv', 'a',newline='') as f_object:
            dictwriter_object = DictWriter(f_object,fieldnames=d.keys(),delimiter='\t')
            dictwriter_object.writerow(d)
            f_object.close()
    except:
        return

def content(data):
    data = data.find('div',class_='lister list detail sub-list').find_all('div',class_='lister-item mode-advanced')
    for link in data:
        link = link.h3.a['href']
        i = "https://imdb.com"+link
        d = extract(i)
        load(d)

def travel(data):
    next_data = data.find('div',class_='desc').find('a',class_='lister-page-next next-page')
    next_link ='https://www.imdb.com'+next_data['href']
    return next_link
            
data = fetch("https://www.imdb.com/search/title/?title_type=feature&release_date=2000-01-01,2009-12-31&languages=en&start=200")
while(data!=None):
    content(data)
    data = fetch(travel(data))

{'title': 'Dodgeball: A True Underdog Story', 'director': 'Rawson Marshall Thurber', 'writer': ['Rawson Marshall Thurbe'], 'cast': ['Vince Vaughn', 'Christine Taylor', 'Ben Stiller', 'Rip Torn', 'Justin Long', 'Stephen Root', 'Joel David Moore', 'Chris Williams', 'Alan Tudyk', 'Missi Pyle', 'Jamal Duff', 'Gary Cole', 'Jason Bateman', 'Hank Azaria', 'Al Kaplon', 'Lance Armstrong', 'Chuck Norris', 'William Shatner', 'David Hasselhoff', 'Julie Gonzalo', "Trever O'Brien", 'Cayden Boyd', 'Rusty Joiner', 'Kevin Porter', 'Brandon Molale', 'Suzy Nakamura', 'Lori Beth Denberg', 'Julia Ensign', 'David Boyd', 'Bowd Beal', 'Curtis Armstrong', 'Tate Chalk', 'Jordyn Colemon', 'Hayley C. Rosales', 'Bix Barnaba', 'Earl Schuman', "Robert 'Duckie' Carpenter", 'Tony Daly', 'Amy Stiller', 'Jim Cody Williams', 'John Kesler', 'Doug Grimes', 'Scarlett Chorvat', 'Matt Levin', 'Rawson Marshall Thurber', 'Sik End', 'Stephen B. Turner', 'Tim Soergel', 'Andy Chanley', 'Jennifer England', 'Priscilla Leona Horne', 

8.7
